In [46]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
import re
from time import sleep
from selenium import webdriver
import numpy as np
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By

In [75]:
# Function to check if the current page is available
def is_page_available(driver):
    try:
        # Wait for an element on the page that indicates its presence
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'product-card--vHfY9')))
        return True
    except TimeoutException:
        return False

In [76]:
# url='https://www.daraz.pk/smartphones/'
# Headers for request
# webpage=requests.get(url,HEADERS)
webpage=[]
driver = webdriver.Chrome()
i=1
while(True):
    url='https://www.daraz.pk/smartphones/'+'?page='+str(i)
    driver.get(url)
    if(is_page_available(driver)):
        webpage.append(driver.page_source)
        i=i+1
    else:
        break
driver.close()

In [77]:
len(webpage)

58

In [73]:
for page in webpage[:5]:
    soup = BeautifulSoup(page, "html.parser")
    links=soup.find_all('a','product-card--vHfY9')
    scrape_data(links)

In [65]:
def initialize_dict():
    dict={'title':[],'price':[],'rating':[]}
    return dict

In [66]:
dict=initialize_dict()

In [67]:
def get_title(soup):
    try:
        title=soup.find('span','pdp-mod-product-badge-title')
        title_string=title.text.strip()
    except AttributeError:
        title_string=""
    return title_string

In [68]:
def get_price(soup):
    try:
        price=soup.find('span','pdp-price pdp-price_type_normal pdp-price_color_orange pdp-price_size_xl')
        price_string=price.text.strip()
    except AttributeError:
        price_string=""
    return price_string

In [69]:
def get_rating(soup):
    try:
        rating=soup.find('a','pdp-link pdp-link_size_s pdp-link_theme_blue pdp-review-summary__link')
        rating_string=rating.text.strip()
    except AttributeError:
        rating_string=""
    return rating_string

In [70]:
def scrape_data(links):
    driver = webdriver.Chrome()
    for page_link in links:
        url=page_link.get('href')
        page_url='https:'+url
        driver.get(page_url)
        webpage = driver.page_source
        soup = BeautifulSoup(webpage, "html.parser")
        dict['title'].append(get_title(soup))
        dict['price'].append(get_price(soup))
        dict['rating'].append(get_rating(soup))
    
    driver.close()

In [235]:
dict

{'title': ['TECNO SPARK Go 2024 64GB ROM+8GB RAM*(4GB+4GB Extended)   6.6"HD+Hole PTA APPROVED OFFICIAL BRAND WARRANTY',
  'Galaxy A04s 4GB RAM-128GB ROM PTA APPROVED NON ACTIVE OFFICIAL BRAND WARRANNTY',
  "Samsung A14 4GB/128GB - 5000 mAh Battery - 6.6' Display - Android 13 PTA Approved Non Active with 1 Year Official Warranty",
  'SPARK 20C  128GB ROM+4GB RAM*(4GB+4GB Extended)  5000mAh(typ) Battery PTA Approved Official Brand Warranty',
  'Samsung Galaxy A04 - 6.5" Inch Display - 3GB RAM - 32GB ROM',
  'Neo 7 Plus | 4GB/64GB | Display: 6.56" | Camera: 13MP Dual Camera | 8MP Selfie | Battery: 5000mAh | Processor: Octa-Core | Sparx',
  'SPARX NEO 7 ULTRA 6+128GB || 6.7" DISPLAY || 5000mAh BATTERY || OCTA CORE PROCESSOR || FAST CHARGING || PTA APPROVED || ONE YEAR BRAND WARRANTY',
  'Infinix Hot 40i || 8GB+8GB Ram 5000mAh Battery || 50MP Camera',
  'Redmi 13C  4GB RAM-128GB ROM  6.74" Dot Drop display Battery & Charging: 5000mAh (typ)  PTA APPROVED OFFICIAL WARRANTY',
  'INFINIX HOT40

In [74]:
daraz_df = pd.DataFrame.from_dict(dict)
daraz_df['title'].replace('', np.nan, inplace=True)
daraz_df = daraz_df.dropna(subset=['title'])
daraz_df.to_csv("daraz_data_phones.csv", header=True, index=False)
dict=initialize_dict()